In [26]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.linear_model import LinearRegression


In [27]:
# directory related variables
DATA_DIR = './data/'
CANCER_DATA = 'cancer_incd_rate_2016_2020.csv'
AQI_DATA = lambda year: f'annual_aqi_by_county_{year}.csv'
US_COUNTIES = 'uscounties/uscounties.csv'

In [31]:
def get_uscounties():
    '''
    Returns 
        a DataFrame of US counties with their FIPS code
    '''
    df = pd.read_csv(DATA_DIR + US_COUNTIES).rename({'county': 'County', 'county_fips': 'FIPS'}, axis=1)
    df["FIPS"] = pd.to_numeric(df["FIPS"], errors='coerce')
    df["FIPS"] = df["FIPS"].astype('Int64')
    return df
get_uscounties().head()

,County,county_ascii,county_full,FIPS,state_id,state_name,lat,lng,population
0,Los Angeles,Los Angeles,Los Angeles County,6037,CA,California,34.3219,-118.2247,9936690
1,Cook,Cook,Cook County,17031,IL,Illinois,41.8401,-87.8168,5225367
2,Harris,Harris,Harris County,48201,TX,Texas,29.8578,-95.3938,4726177
3,Maricopa,Maricopa,Maricopa County,4013,AZ,Arizona,33.3490,-112.4915,4430871
4,San Diego,San Diego,San Diego County,6073,CA,California,33.0343,-116.7351,3289701


In [32]:
def replace_nan_variants(df: pd.DataFrame):
    '''
    replaces common nan variants with np.nan in a pandas DataFrame
    replaced '3 or fewer' with '3'
    Args:
        df: a pandas DataFrame
    Returns:
        a clean pandas dataframe

    '''
    df = df.replace('data not available', np.nan)
    df = df.replace('data not available ', np.nan)
    df = df.replace(' data not available', np.nan)
    df = df.replace('*', np.nan)
    df = df.replace('* ', np.nan)
    df = df.replace(' *', np.nan)
    df = df.replace(' * ', np.nan)
    df = df.replace('3 or fewer', '3')
    return df
    

def convert_to_numeric(df: pd.DataFrame):
    '''
    Converts specific columns in a pandas DataFrame to numeric
    Args:
        df: a pandas DataFrame
    Returns:
        a pandas DataFrame with specified columns converted to numeric
    '''
    df['Recent 5-Year Trend'] = pd.to_numeric(df['Recent 5-Year Trend'], errors='coerce')
    df['Incidence Rate per 100k'] = pd.to_numeric(df['Incidence Rate per 100k'], errors='coerce')
    df['Lower 95% Confidence Interval'] = pd.to_numeric(df['Lower 95% Confidence Interval'], errors='coerce')
    df['Upper 95% Confidence Interval'] = pd.to_numeric(df['Upper 95% Confidence Interval'], errors='coerce')
    df['Average Annual Count'] = pd.to_numeric(df['Average Annual Count'], errors='coerce')
    df['FIPS'] = pd.to_numeric(df['FIPS'], errors='coerce')
    df["FIPS"] = df["FIPS"].astype('Int64')
    return df

def merge_locational_data(df: pd.DataFrame):
    '''
    Merges a pandas DataFrame with locational data
    Args:
        df: a pandas DataFrame
    Returns:
        a pandas DataFrame with locational data merged into it
    '''
    coords_df = get_uscounties()
    df = df.merge(
        coords_df, on='FIPS'
    ).drop(
        columns=['County_x', 'state_id', 'county_full','county_ascii', 'State']
    ).rename(
        {'County_y': 'County', 'state_name': 'State', 'lat': 'Lat', 'lng': 'Lon', 'population': 'Population'}, 
        axis=1
    )
    return df

def get_cancer_data():
    '''
    Returns 
        a DataFrame of cancer data for 2016-2020
    '''
    df = pd.read_csv(DATA_DIR + CANCER_DATA, skiprows=8, skipfooter=31, engine='python')
    df.insert(0, 'State', df['County'].apply(lambda x: x.split(', ')[-1][:-3]))
    df['County'] = df['County'].apply(lambda x: x.split(', ')[0])
    df = df.iloc[1:].rename({
        'Age-Adjusted Incidence Rate([rate note]) - cases per 100,000': 'Incidence Rate per 100k', 
        'Recent 5-Year Trend ([trend note]) in Incidence Rates': 'Recent 5-Year Trend',
        ' FIPS': 'FIPS'
    }, axis=1)

    df = df.pipe(replace_nan_variants).pipe(convert_to_numeric).pipe(merge_locational_data)
    # ignore nan values for recent 5-year trend
    df = df[df['Recent 5-Year Trend'].notna()]
    return df
get_cancer_data().head()

,FIPS,Incidence Rate per 100k,Lower 95% Confidence Interval,Upper 95% Confidence Interval,CI*Rank([rank note]),Lower CI (CI*Rank),Upper CI (CI*Rank),Average Annual Count,Recent Trend,Recent 5-Year Trend,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1,County,State,Lat,Lon,Population
0,12125,1237.4,1165.6,1312.8,N/A,1,1,237.0,stable,0.6,-0.5,1.9,Union,Florida,30.0439,-82.3714,15524
1,19147,658.1,591.1,731.1,N/A,1,6,82.0,rising,4.8,0.2,15.4,Palo Alto,Iowa,43.0821,-94.6781,8938
2,30103,652.2,401.0,1007.4,N/A,1,55,7.0,stable,-1.1,-5.6,3.3,Treasure,Montana,46.2115,-107.2716,680
3,48373,633.6,604.6,663.7,N/A,1,4,425.0,rising,2.2,1.2,4.2,Polk,Texas,30.7927,-94.8300,50536
4,21071,616.8,584.3,650.7,N/A,1,19,295.0,stable,1.5,-1.8,5.3,Floyd,Kentucky,37.5571,-82.7457,35780


In [33]:
def merge_locational_data_2(df: pd.DataFrame):
    '''
    similar to merge_locational_data but merges on 'County' instead of 'FIPS', and less necessary cleaning
    Args:
        df: a pandas DataFrame
    Returns:
        a pandas DataFrame with locational data merged into it
    '''
    coords_df = get_uscounties()
    df = df.merge(
        coords_df, on=['County']
    ).drop_duplicates(subset=['County'],keep='first')
    
    return df

def get_aqi_data():
    '''
    compile aqi data from 2016-2020 into a single DataFrame
    Returns:
        a DataFrame of aqi data for 2016-2020
    '''
    
    # combine years aqi dfs
    dfs = []
    for year in range(2016, 2021):
        aqi = pd.read_csv(DATA_DIR + AQI_DATA(year))
        dfs.append(aqi)
    annual_aqi_2016_2020 = pd.concat(dfs)

    
    trend_dfs = annual_aqi_2016_2020.copy()
    # get average/sum of data across all years, grouping by county
    annual_aqi_2016_2020 = annual_aqi_2016_2020.groupby(['State', 'County']).agg(
        {
            'Good Days': 'sum', 
            'Moderate Days': 'sum', 
            'Unhealthy for Sensitive Groups Days': 'sum',
            'Unhealthy Days': 'sum',
            'Very Unhealthy Days': 'sum',
            'Hazardous Days': 'sum',
            'Max AQI': 'max',
            'Median AQI': 'mean',
            'Days CO': 'sum',
            'Days NO2': 'sum',
            'Days Ozone': 'sum',
            'Days PM2.5': 'sum',
            'Days PM10': 'sum',
        }
    ).reset_index()

    # add recent 5-year trend
    trend_df = trend_dfs.groupby(['State', 'County']).apply(lambda x: x['Median AQI'].pct_change())

    # clean data
    annual_aqi_2016_2020 = annual_aqi_2016_2020.pipe(merge_locational_data_2).merge(
        trend_df, on=['State', 'County']
    ).rename({'Median AQI_x': 'Median AQI', 'Median AQI_y': 'Recent 5-Year Trend'}, axis=1)
    
    return annual_aqi_2016_2020

get_aqi_data().head()

,State,County,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,Median AQI,...,Days PM10,county_ascii,county_full,FIPS,state_id,state_name,lat,lng,population,Recent 5-Year Trend
0,Alabama,Baldwin,1220,138,1,0,0,0,108,36.2,...,0,Baldwin,Baldwin County,1003,AL,Alabama,30.7277,-87.7226,233420,NaN
1,Alabama,Baldwin,1220,138,1,0,0,0,108,36.2,...,0,Baldwin,Baldwin County,1003,AL,Alabama,30.7277,-87.7226,233420,-0.027027
2,Alabama,Baldwin,1220,138,1,0,0,0,108,36.2,...,0,Baldwin,Baldwin County,1003,AL,Alabama,30.7277,-87.7226,233420,-0.027778
3,Alabama,Baldwin,1220,138,1,0,0,0,108,36.2,...,0,Baldwin,Baldwin County,1003,AL,Alabama,30.7277,-87.7226,233420,0.057143
4,Alabama,Baldwin,1220,138,1,0,0,0,108,36.2,...,0,Baldwin,Baldwin County,1003,AL,Alabama,30.7277,-87.7226,233420,-0.027027


# Cancer EDA

In [6]:
# compare incident rate with 5 year trend
cancer_df = get_cancer_data()
fig = px.scatter(
    cancer_df, 
    x='Incidence Rate per 100k', 
    y='Recent 5-Year Trend', 
    hover_name='County', 
)
fig.show()

In [7]:
# get incident rate distribution
fig = px.histogram(
    cancer_df, 
    x='Incidence Rate per 100k', 
    nbins=100, 
    title='Cancer Incidence Rate per 100k Distribution'
)
fig.show()

In [8]:
# get cancer 5-year trend distribution
fig = px.histogram(
    cancer_df, 
    x='Recent 5-Year Trend', 
    nbins=100, 
    title='Cancer Trend Distribution'
)
fig.show()

In [9]:

# get geo map of average annual incident rate count
# normalize trend data
min_trend_value = cancer_df['Recent 5-Year Trend'].min()
max_trend_value = cancer_df['Recent 5-Year Trend'].max()
cancer_df['Recent 5-Year Trend'] = cancer_df['Recent 5-Year Trend'].apply(lambda x: (x-min_trend_value)/(max_trend_value-min_trend_value))
cancer_df
# geo map of cancer data
fig = px.scatter_geo(
    cancer_df, 
    lat='Lat', 
    lon='Lon', 
    hover_name='County', 
    hover_data=['Incidence Rate per 100k', 'Recent 5-Year Trend', 'Population'],
    size='Average Annual Count', 
    title='Average Annual Incident Rate Count',
    color='Recent 5-Year Trend',
    projection='albers usa',
    height=500,
    width=1000
)
fig.update_layout(
    title_x=0.5
)
fig.show()

-26.9 21.4


In [10]:

cancer_df = get_cancer_data()
fig = px.scatter_geo(
    cancer_df, 
    lat='Lat', 
    lon='Lon', 
    hover_name='County', 
    hover_data=['Incidence Rate per 100k', 'Recent 5-Year Trend', 'Population'],
    color='Recent 5-Year Trend', 
    projection='albers usa',
    width=1000,
    height=500
)
fig.update_layout(
    title_text='Recent 5-Year Cancer Trend',
    title_x=0.5
)

fig.show()

In [11]:
cancer_df = get_cancer_data()
avg_cancer_df = cancer_df.groupby('State').agg(
    {
        'Incidence Rate per 100k': 'mean',
        'Recent 5-Year Trend': 'mean'
    }
).reset_index()
fig = px.bar(
    avg_cancer_df, 
    x='State', 
    y='Recent 5-Year Trend', 
    title='Recent 5-Year Trend by State'
)
fig.show()

In [12]:
fig = px.bar(
    avg_cancer_df,
    x='State', 
    y='Incidence Rate per 100k', 
    title='Incidence Rate per 100k v State',
    color='Recent 5-Year Trend',
)
fig.update_layout(
    title_x=0.5,
    xaxis={'categoryorder':'total descending'}
)
fig.show()

# AQI EDA

In [13]:
aqi_df = get_aqi_data()
fig = px.scatter_geo(
    aqi_df, 
    lat='lat', 
    lon='lng', 
    hover_name='County', 
    hover_data=['Median AQI', 'Max AQI'],
    color='Median AQI', 
    projection='albers usa',
    title='Median AQI by County',
    width=1000,
    height=500
)
fig.update_layout(
    title_x=0.5
)
fig.show()

In [14]:
aqi_df = get_aqi_data()
fig = px.scatter_geo(
    aqi_df,
    lat='lat', 
    lon='lng', 
    hover_name='County', 
    hover_data=['Median AQI', 'Max AQI'],
    color='Recent 5-Year Trend', 
    projection='albers usa',
    title='Recent 5-Year AQI Trend by County',
    width=1000,
    height=500
)
fig.update_layout(
    title_x=0.5
)
fig.show()

In [15]:
aqi_df = get_aqi_data()
aqi_df

,State,County,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,Median AQI,...,Days PM10,county_ascii,county_full,FIPS,state_id,state_name,lat,lng,population,Recent 5-Year Trend
0,Alabama,Baldwin,1220,138,1,0,0,0,108,36.2,...,0,Baldwin,Baldwin County,1003,AL,Alabama,30.7277,-87.7226,233420,NaN
1,Alabama,Baldwin,1220,138,1,0,0,0,108,36.2,...,0,Baldwin,Baldwin County,1003,AL,Alabama,30.7277,-87.7226,233420,-0.027027
2,Alabama,Baldwin,1220,138,1,0,0,0,108,36.2,...,0,Baldwin,Baldwin County,1003,AL,Alabama,30.7277,-87.7226,233420,-0.027778
3,Alabama,Baldwin,1220,138,1,0,0,0,108,36.2,...,0,Baldwin,Baldwin County,1003,AL,Alabama,30.7277,-87.7226,233420,0.057143
4,Alabama,Baldwin,1220,138,1,0,0,0,108,36.2,...,0,Baldwin,Baldwin County,1003,AL,Alabama,30.7277,-87.7226,233420,-0.027027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3704,Wyoming,Weston,1522,175,2,0,0,0,119,40.8,...,0,Weston,Weston County,56045,WY,Wyoming,43.8404,-104.5676,6870,NaN
3705,Wyoming,Weston,1522,175,2,0,0,0,119,40.8,...,0,Weston,Weston County,56045,WY,Wyoming,43.8404,-104.5676,6870,0.000000
3706,Wyoming,Weston,1522,175,2,0,0,0,119,40.8,...,0,Weston,Weston County,56045,WY,Wyoming,43.8404,-104.5676,6870,0.024390
3707,Wyoming,Weston,1522,175,2,0,0,0,119,40.8,...,0,Weston,Weston County,56045,WY,Wyoming,43.8404,-104.5676,6870,-0.047619


In [16]:
cancer = get_cancer_data()
merged = cancer.merge(aqi_df, on='FIPS', how='inner')
merged

,FIPS,Incidence Rate per 100k,Lower 95% Confidence Interval,Upper 95% Confidence Interval,CI*Rank([rank note]),Lower CI (CI*Rank),Upper CI (CI*Rank),Average Annual Count,Recent Trend,Recent 5-Year Trend_x,...,Days PM2.5,Days PM10,county_ascii,county_full,state_id,state_name,lat,lng,population,Recent 5-Year Trend_y
0,19147,658.1,591.1,731.1,N/A,1,6,82.0,rising,4.8,...,1019,3,Palo Alto,Palo Alto County,IA,Iowa,43.0821,-94.6781,8938,NaN
1,19147,658.1,591.1,731.1,N/A,1,6,82.0,rising,4.8,...,1019,3,Palo Alto,Palo Alto County,IA,Iowa,43.0821,-94.6781,8938,0.111111
2,19147,658.1,591.1,731.1,N/A,1,6,82.0,rising,4.8,...,1019,3,Palo Alto,Palo Alto County,IA,Iowa,43.0821,-94.6781,8938,0.050000
3,19147,658.1,591.1,731.1,N/A,1,6,82.0,rising,4.8,...,1019,3,Palo Alto,Palo Alto County,IA,Iowa,43.0821,-94.6781,8938,-0.095238
4,19147,658.1,591.1,731.1,N/A,1,6,82.0,rising,4.8,...,1019,3,Palo Alto,Palo Alto County,IA,Iowa,43.0821,-94.6781,8938,-0.052632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3381,26083,240.0,171.7,336.7,N/A,74,83,12.0,stable,-1.2,...,589,0,Keweenaw,Keweenaw County,MI,Michigan,47.6279,-88.4346,2088,NaN
3382,26083,240.0,171.7,336.7,N/A,74,83,12.0,stable,-1.2,...,589,0,Keweenaw,Keweenaw County,MI,Michigan,47.6279,-88.4346,2088,0.111111
3383,26083,240.0,171.7,336.7,N/A,74,83,12.0,stable,-1.2,...,589,0,Keweenaw,Keweenaw County,MI,Michigan,47.6279,-88.4346,2088,-0.100000
3384,26083,240.0,171.7,336.7,N/A,74,83,12.0,stable,-1.2,...,589,0,Keweenaw,Keweenaw County,MI,Michigan,47.6279,-88.4346,2088,0.000000


In [17]:

px.scatter(merged, x="Unhealthy Days", y="Recent 5-Year Trend", color="Recent Trend")

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['FIPS', 'Incidence Rate per 100k', 'Lower 95% Confidence Interval', 'Upper 95% Confidence Interval', 'CI*Rank([rank note])', 'Lower CI (CI*Rank)', 'Upper CI (CI*Rank)', 'Average Annual Count', 'Recent Trend', 'Recent 5-Year Trend_x', 'Lower 95% Confidence Interval.1', 'Upper 95% Confidence Interval.1', 'County_x', 'State_x', 'Lat', 'Lon', 'Population', 'State_y', 'County_y', 'Good Days', 'Moderate Days', 'Unhealthy for Sensitive Groups Days', 'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI', 'Median AQI', 'Days CO', 'Days NO2', 'Days Ozone', 'Days PM2.5', 'Days PM10', 'county_ascii', 'county_full', 'state_id', 'state_name', 'lat', 'lng', 'population', 'Recent 5-Year Trend_y'] but received: Recent 5-Year Trend

In [ ]:
px.scatter(merged, x="Median AQI", y="Recent 5-Year Trend", color="Recent Trend")

In [ ]:
for i in ["Days Ozone", "Days PM2.5", "Days PM10", "Days CO", "Median AQI"]:
    px.scatter(merged, x=i, y="Average Annual Count", color="Recent Trend").show() 

In [ ]:
for i in ["Moderate Days", "Unhealthy for Sensitive Groups Days", "Unhealthy Days", "Very Unhealthy Days", "Hazardous Days"]:
    px.scatter(merged, x=i, y="Average Annual Count", color="Recent Trend").show() 


In [ ]:
px.scatter(merged, x="Moderate Days", y="Average Annual Count").show()

In [ ]:
merged.corr()

,FIPS,Incidence Rate per 100k,Lower 95% Confidence Interval,Upper 95% Confidence Interval,Average Annual Count,Recent 5-Year Trend,Lat,Lon,Population,Good Days,...,Max AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,lat,lng,population
FIPS,1.000000,0.095451,0.077178,0.110483,-0.111376,0.078429,0.290161,0.163348,-0.099661,0.112590,...,-0.078720,-0.190052,0.037576,0.008103,0.004089,0.003550,-0.108712,0.290161,0.163348,-0.099661
Incidence Rate per 100k,0.095451,1.000000,0.971867,0.957149,-0.020146,0.384472,0.029700,0.472709,-0.076558,0.158112,...,-0.223813,-0.018603,0.051233,-0.039382,0.077371,0.011973,-0.161490,0.029700,0.472709,-0.076558
Lower 95% Confidence Interval,0.077178,0.971867,1.000000,0.862339,0.088432,0.362607,-0.010945,0.482751,0.026156,0.170548,...,-0.218515,0.054817,0.052702,-0.007159,0.101128,0.034162,-0.169258,-0.010945,0.482751,0.026156
Upper 95% Confidence Interval,0.110483,0.957149,0.862339,1.000000,-0.143875,0.382229,0.077602,0.427746,-0.189747,0.130583,...,-0.214601,-0.104635,0.045747,-0.074417,0.043276,-0.014818,-0.142323,0.077602,0.427746,-0.189747
Average Annual Count,-0.111376,-0.020146,0.088432,-0.143875,1.000000,-0.028390,-0.115441,0.040452,0.983077,-0.112035,...,0.014094,0.401733,-0.000617,0.213155,0.040121,0.098739,-0.040442,-0.115441,0.040452,0.983077
Recent 5-Year Trend,0.078429,0.384472,0.362607,0.382229,-0.028390,1.000000,-0.111473,0.074553,-0.023043,0.008408,...,-0.053495,0.016504,0.001372,-0.005091,0.049379,-0.030617,-0.063751,-0.111473,0.074553,-0.023043
Lat,0.290161,0.029700,-0.010945,0.077602,-0.115441,-0.111473,1.000000,-0.094816,-0.117243,0.084271,...,0.080797,-0.117214,-0.005415,-0.001084,-0.081052,0.127658,0.019558,1.000000,-0.094816,-0.117243
Lon,0.163348,0.472709,0.482751,0.427746,0.040452,0.074553,-0.094816,1.000000,-0.014654,0.097243,...,-0.252588,0.008312,0.034978,-0.019705,0.187104,-0.219613,-0.133114,-0.094816,1.000000,-0.014654
Population,-0.099661,-0.076558,0.026156,-0.189747,0.983077,-0.023043,-0.117243,-0.014654,1.000000,-0.144705,...,0.023811,0.417136,-0.005840,0.221864,0.032295,0.092703,-0.033305,-0.117243,-0.014654,1.000000
Good Days,0.112590,0.158112,0.170548,0.130583,-0.112035,0.008408,0.084271,0.097243,-0.144705,1.000000,...,-0.001785,-0.121147,0.073570,0.016057,0.413713,0.179299,0.046935,0.084271,0.097243,-0.144705


In [ ]:
merged.columns

Index(['FIPS', 'Incidence Rate per 100k', 'Lower 95% Confidence Interval',
       'Upper 95% Confidence Interval', 'CI*Rank([rank note])',
       'Lower CI (CI*Rank)', 'Upper CI (CI*Rank)', 'Average Annual Count',
       'Recent Trend', 'Recent 5-Year Trend',
       'Lower 95% Confidence Interval.1', 'Upper 95% Confidence Interval.1',
       'County_x', 'State_x', 'Lat', 'Lon', 'Population', 'State_y',
       'County_y', 'Good Days', 'Moderate Days',
       'Unhealthy for Sensitive Groups Days', 'Unhealthy Days',
       'Very Unhealthy Days', 'Hazardous Days', 'Max AQI', 'Median AQI',
       'Days CO', 'Days NO2', 'Days Ozone', 'Days PM2.5', 'Days PM10',
       'county_ascii', 'county_full', 'state_id', 'state_name', 'lat', 'lng',
       'population'],
      dtype='object')

# Linear Regression

In [38]:
# 
cancer_df = get_cancer_data()
aqi_df = get_aqi_data()
county_aqi_and_cancer_trends = cancer_df.merge(aqi_df, on='FIPS', how='inner').rename(
    {
        'Recent 5-Year Trend_x': 'Cancer Trend',
        'Recent 5-Year Trend_y': 'AQI Trend'
    }, axis=1
)
county_aqi_and_cancer_trends = county_aqi_and_cancer_trends[county_aqi_and_cancer_trends['AQI Trend'].notna()]
regressor = LinearRegression()
X = county_aqi_and_cancer_trends[['Median AQI']]
y = county_aqi_and_cancer_trends['Incidence Rate per 100k']
s = regressor.fit(X, y)
regressor.score(X, y)


AttributeError: 'LinearRegression' object has no attribute 'summary'